# Test Article

Загружаем библиотеки

In [1]:
from transformers import AutoTokenizer, AutoModel, pipeline, logging

In [2]:
modelpath = "../model/fine-train"
# modelpath = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
model = AutoModel.from_pretrained(modelpath)

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /ai-forever/ruBert-base/resolve/main/tokenizer_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1007)')))"), '(Request ID: 9e59b743-c794-499b-99df-45d35fd14a0e)')

In [ ]:
text = '''Ранее в четверг лидеры Европейского союз единогласно согласовали выделение Украине макрофинансовой помощи в размере 50 миллиардов евро на период до 2027 года с условием ежегодной отчетности со стороны ЕК и дебатов в Евросовете. Как сообщил премьер-министр Польши Дональд Туск, главу венгерского правительства Виктора Орбана "удалось убедить".
Позиция Будапешта оставалась главным препятствием для выделения денег киевским властям. Как выяснила газета Financial Times, лидеры стран Евросоюза могли публично объявить на саммите о неконструктивном поведении Орбана и отказать Венгрии в разморозке средств из европейских фондов.
Как подчеркивал сам венгерский премьер, такое развитие событий привело бы к армагеддону. Он заявил, что европейцам 50 миллиардов евро нужны не меньше, чем украинцам, но предложил компромисс: не выделять всю сумму сразу на четыре года, а утверждать финансирование для Киева ежегодно и единогласно.'''

In [ ]:
import re

tr = 0.0001
logging.set_verbosity_error()

def mask_bert_sent(text, model, tokenizer):
    unmasker = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    p = re.compile(r'[\w-]+')
    iter = p.finditer(text)
    listErr = []
    for match in iter:
        if match.start() == 0:
            masktext = "[MASK]" + text[match.end():]
        elif match.end() == len(text):
            masktext = text[:match.start()] + "[MASK]"
        else:
            masktext = text[:match.start()] + "[MASK]" + text[match.end():]
        
        res = unmasker(masktext, targets=[match.group()])
        if res[0]['score'] < tr:
            errorrDesc = {
                "word": match.group(),
                "start": match.start(),
                "end": match.end(),
                "prob": res[0]['score']
            }
            listErr.append(errorrDesc)
    return listErr


In [ ]:
text = text.casefold()
errors = mask_bert_sent(text, modelpath, tokenizer)
if len(errors) > 0:
    for e in errors:
        print (e)

{'word': 'европейского', 'start': 23, 'end': 35, 'prob': 9.819237675401382e-06}
{'word': 'союз', 'start': 36, 'end': 40, 'prob': 6.908896466484293e-06}
{'word': 'единогласно', 'start': 41, 'end': 52, 'prob': 1.4520525155603536e-06}
{'word': 'ек', 'start': 201, 'end': 203, 'prob': 5.047909780842019e-06}
{'word': 'виктора', 'start': 309, 'end': 316, 'prob': 1.102602527680574e-05}
{'word': 'орбана', 'start': 317, 'end': 323, 'prob': 1.9512188373482786e-05}
{'word': 'будапешта', 'start': 351, 'end': 360, 'prob': 1.720545333228074e-05}
{'word': 'неконструктивном', 'start': 528, 'end': 544, 'prob': 2.6694253392633982e-05}
{'word': 'орбана', 'start': 555, 'end': 561, 'prob': 1.043819821688885e-07}
{'word': 'венгрии', 'start': 573, 'end': 580, 'prob': 1.319154421253188e-06}
{'word': 'разморозке', 'start': 583, 'end': 593, 'prob': 1.726279919012086e-08}
{'word': 'армагеддону', 'start': 701, 'end': 712, 'prob': 4.920724383516983e-10}
